# ``jenga`` Corruptions

This notebook illustrates some of the corruptions used in jenga

Most corruptions are operating on tabular data, for which different sampling mechanisms are used. 

Inspired by research on missing values, we consider three different sampling schemes for corrupting tabular data (we'll keep using the 'missingness' terminology here, even if we use other corruptions than discarding data):

* **MCAR** Missing completely at random
* **MAR** Missing at random - corruption is conditioned on other column
* **MNAR** Missing not at random - corruption is conditioned on values in column on which it is applied


In [1]:
%load_ext autoreload
%autoreload 2

import random
import pandas as pd
import numpy as np

from jenga.corruptions.generic import MissingValues

In [2]:
def new_df(N=20):
    return pd.DataFrame({
        'A':np.arange(0,N) / N, 
        'B': [random.choice(['a','b','c']) for _ in range(N)],
        'C': [random.choice(['foo','bah']) for _ in range(N)]
        })

## Missing Values

Below some examples on different row sampling strategies for corruptions with missing values

In [3]:
from jenga.corruptions.generic import MissingValues

df = new_df()

df['B_MCAR'] = MissingValues(column='B', fraction=.5, missingness='MCAR').transform(df)['B']
df['B_MAR'] = MissingValues(column='B', fraction=.5, missingness='MAR').transform(df)['B']
df['B_MNAR'] = MissingValues(column='B', fraction=.5, missingness='MNAR').transform(df)['B']

df

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,c,foo,NaN,NaN,NaN
1,0.05,a,foo,a,a,a
2,0.10,b,foo,b,b,NaN
3,0.15,a,bah,a,a,a
4,0.20,b,foo,NaN,NaN,NaN
5,0.25,a,foo,NaN,NaN,a
6,0.30,b,foo,b,NaN,NaN
7,0.35,c,bah,NaN,NaN,c
8,0.40,a,foo,a,NaN,a
9,0.45,c,foo,c,c,c


If we sort according to the values in the column on which the corruption as applied, we see that the 'MNAR' condition works as it should

In [4]:
df.sort_values('B')

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.00,a,bah,NaN,a,a
16,0.80,a,bah,NaN,NaN,a
5,0.25,a,bah,NaN,NaN,NaN
6,0.30,a,foo,a,a,NaN
18,0.90,a,foo,a,a,NaN
15,0.75,b,bah,NaN,NaN,NaN
14,0.70,b,foo,b,b,NaN
10,0.50,b,bah,NaN,NaN,NaN
19,0.95,b,bah,NaN,NaN,NaN
2,0.10,b,bah,NaN,NaN,NaN


## Swapping Values

Also works between numeric and non-numeric values, with different sampling schemes

In [5]:
from jenga.corruptions.generic import SwappedValues
df = new_df()
df['C_swapped_MAR'] = SwappedValues(column='B', fraction=.5, sampling='MAR').transform(df)['C']
df['C_swapped_MCAR'] = SwappedValues(column='B', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_swapped_MCAR'] = SwappedValues(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,C_swapped_MAR,C_swapped_MCAR,A_swapped_MCAR
0,0.00,a,foo,foo,foo,0.0
1,0.05,b,foo,b,foo,b
2,0.10,b,bah,b,bah,0.1
3,0.15,a,bah,a,bah,0.15
4,0.20,b,foo,b,foo,b
5,0.25,b,foo,b,foo,0.25
6,0.30,c,foo,c,foo,0.3
7,0.35,c,bah,c,bah,c
8,0.40,a,foo,a,foo,0.4
9,0.45,a,bah,bah,bah,a


## Messing up categorical Values

These corruptions permute the histogram of values

In [5]:
from jenga.corruptions.generic import CategoricalShift
df = new_df()
df['C_permuted_MAR'] = CategoricalShift(column='C', fraction=.5, sampling='MAR').transform(df)['C']
df['C_permuted_MCAR'] = CategoricalShift(column='C', fraction=.5, sampling='MCAR').transform(df)['C']
df['A_permuted_MCAR'] = CategoricalShift(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

CategoricalShift implemented only for categorical variables


,A,B,C,C_permuted_MAR,C_permuted_MCAR,A_permuted_MCAR
0,0.00,a,bah,bah,foo,0.00
1,0.05,a,bah,bah,bah,0.05
2,0.10,c,bah,bah,bah,0.10
3,0.15,c,bah,bah,foo,0.15
4,0.20,c,bah,bah,bah,0.20
5,0.25,b,foo,foo,bah,0.25
6,0.30,c,bah,foo,foo,0.30
7,0.35,c,foo,bah,foo,0.35
8,0.40,a,bah,foo,bah,0.40
9,0.45,c,foo,bah,foo,0.45


## Adding Gaussian Noise



In [7]:
from jenga.corruptions.numerical import GaussianNoise
df = new_df()
df['A_GaussNoise_MAR'] = GaussianNoise(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_GaussNoise__MCAR'] = GaussianNoise(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_GaussNoise_MAR,A_GaussNoise__MCAR
0,0.00,c,foo,0.000000,0.375077
1,0.05,a,bah,0.050000,0.580530
2,0.10,a,foo,0.100000,0.100000
3,0.15,b,bah,-2.427858,0.082609
4,0.20,b,foo,1.114760,0.200000
5,0.25,a,bah,0.030722,-0.635990
6,0.30,c,foo,0.300000,0.300000
7,0.35,b,foo,-0.675669,0.350000
8,0.40,c,bah,0.400000,0.400000
9,0.45,c,foo,0.450000,0.083701


## Scaling numeric values


In [8]:
from jenga.corruptions.numerical import Scaling
df = new_df()
df['A_Scaled_MAR'] = Scaling(column='A', fraction=.5, sampling='MAR').transform(df)['A']
df['A_Scaled_MCAR'] = Scaling(column='A', fraction=.5, sampling='MCAR').transform(df)['A']
df

,A,B,C,A_Scaled_MAR,A_Scaled_MCAR
0,0.00,c,bah,0.00,0.00
1,0.05,a,bah,50.00,0.05
2,0.10,a,bah,100.00,10.00
3,0.15,c,foo,0.15,15.00
4,0.20,c,bah,200.00,0.20
5,0.25,a,foo,0.25,25.00
6,0.30,b,bah,0.30,0.30
7,0.35,a,foo,0.35,0.35
8,0.40,c,foo,0.40,40.00
9,0.45,c,bah,450.00,0.45


## When all rows are corrupted ...

... the corruption pattern does not matter.

In [4]:
from jenga.corruptions.generic import MissingValues

df = new_df(N=5)

df['B_MCAR'] = MissingValues(column='B', fraction=1., missingness='MCAR').transform(df)['B']
df['B_MAR'] = MissingValues(column='B', fraction=1., missingness='MAR').transform(df)['B']
df['B_MNAR'] = MissingValues(column='B', fraction=1., missingness='MNAR').transform(df)['B']

df

,A,B,C,B_MCAR,B_MAR,B_MNAR
0,0.0,a,foo,NaN,NaN,NaN
1,0.2,a,foo,NaN,NaN,NaN
2,0.4,c,foo,NaN,NaN,NaN
3,0.6,a,bah,NaN,NaN,NaN
4,0.8,b,bah,NaN,NaN,NaN
